In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import statsmodels.api as sm
import re

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("tab10")

In [ ]:
# Create a connection to the SQLite database
conn = sqlite3.connect('data/plots.db')

# Define the SQL query to get comprehensive dataset for analysis
all_data_query = """
SELECT 
    p.doi,
    p.figure_number,
    s.sandstone_name,
    dp.p_mpa,
    dp.q_mpa
FROM data_points dp
JOIN sandstones s ON dp.sandstone_id = s.id
JOIN plots p ON s.plot_id = p.id
ORDER BY p.doi, s.sandstone_name, dp.p_mpa
"""

In [ ]:
# Convert the SQL query result into a DataFrame
df_all_data = pd.read_sql_query(all_data_query, conn)

# Display the DataFrame 
df_all_data

In [ ]:
# Filter for Rutter and Glover Figure 8 data
rut_glov_fig8 = df_all_data[df_all_data['doi'] == '10.1016/j.jsg.2012.08.014']

# Filter for friction data
friction_data = rut_glov_fig8[rut_glov_fig8['sandstone_name'] == 'Porous sandstone friction data']

# Remove Serpentinite data
rut_glov_fig8 = rut_glov_fig8[(rut_glov_fig8['sandstone_name'] != 'Serpentinite (20%)') & (rut_glov_fig8['sandstone_name'] != 'Porous sandstone friction data')]

In [ ]:
# Visualise the sandstone data
plt.figure(figsize=(10, 6))
sns.scatterplot(data=rut_glov_fig8, x='p_mpa', y='q_mpa', hue='sandstone_name')
sns.regplot(data=rut_glov_fig8, x='p_mpa', y='q_mpa', scatter=False, 
            color='black', ci=None, label='Regression Line', 
            line_kws={'linewidth': 1.0, 'linestyle': '--'})
plt.xlabel('P (MPa)', fontsize=10)
plt.ylabel('Q (MPa)', fontsize=10)
plt.xlim(0,1000)
plt.ylim(0,1000)    
plt.title('Rutter and Glover, 2012 Figure 8 Sandstone Data')
plt.legend(loc='lower right', title='Sandstones')
plt.savefig("stats_plots/Rutter_Glover_2012_Fig8.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# OLS Regression
X = rut_glov_fig8['p_mpa']
y = rut_glov_fig8['q_mpa']
X = sm.add_constant(X)  # Adds a constant term to the predictor
model = sm.OLS(y, X).fit()
print(f"OLS Regression Results for Rutter and Glover Figure 8 Sandstone Data \n")
print(model.summary())

In [ ]:
# Create a regression plot for the friction data
plt.figure(figsize=(10, 6))
sns.scatterplot(data=friction_data, x='p_mpa', y='q_mpa')
sns.regplot(data=friction_data, x='p_mpa', y='q_mpa', scatter=False, 
            color='black', ci=None, label='Regression Line', 
            line_kws={'linewidth': 1.0, 'linestyle': '--'})
plt.xlim(0, 700)
plt.ylim(0, 700)
plt.xlabel('P (MPa)', fontsize=10)
plt.ylabel('Q (MPa)', fontsize=10)
plt.title('Regression Plot for Rutter and Glover 2012 Figure 8 Friction Data')
plt.savefig("stats_plots/Rutter_Glover_2012_Fig8_FrictionData.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# OLS for friction data
friction_X = friction_data['p_mpa']
friction_y = friction_data['q_mpa']
friction_X = sm.add_constant(friction_X)  # Adds a constant term to the predictor
friction_model = sm.OLS(friction_y, friction_X).fit()
print(f"OLS Regression Results for Rutter and Glover 2012 Friction Data \n")
print(friction_model.summary())

## Include Wong & Baud, 2012. Figures 5a,b and 6a,b (Limestones)

In [ ]:
# Wong and Baud Data
wong_baud = df_all_data[df_all_data['doi'] == '10.1016/j.jsg.2012.07.010']

# Combine (Wong and Baud) sandstone data with (Rutter and Glover) sandstone data
wong_baud_fig5 = wong_baud[wong_baud['figure_number'].isin(['5a', '5b'])]

rutter_glover_wong_baud = pd.concat([rut_glov_fig8, wong_baud_fig5], ignore_index=True)

# Wong and Baud Limestone Data
wong_baud_limestone = wong_baud[wong_baud['figure_number'].isin(['6a', '6b'])]

In [ ]:
# Create a regression plot for new data including Wong and Baud, 2012 data

# Sort the sandstone names for consistent ordering
sandstones = sorted(rutter_glover_wong_baud['sandstone_name'].unique())
n_groups = len(sandstones)

# Pairing colors and markers
colors = sns.color_palette("colorblind", n_colors=min(n_groups, 8))
markers = ['o', 's', 'D', '^', 'v', 'P', 'X', '<', '>'] 

fig, ax = plt.subplots(figsize=(10, 6))

# Plot each group separately
for i, name in enumerate(sandstones):
    group = rutter_glover_wong_baud[rutter_glover_wong_baud['sandstone_name'] == name]
    ax.scatter(
        group['p_mpa'], group['q_mpa'],
        label=name,
        color=colors[i % len(colors)],
        marker=markers[i % len(markers)],
        edgecolor='black',
        linewidth=0.4,
        s=40,
        alpha=0.8
    )

# Regression line
sns.regplot(
    data=rutter_glover_wong_baud, x='p_mpa', y='q_mpa',
    scatter=False, ci=None,
    line_kws={'color': 'black', 'linewidth': 1.0, 'linestyle': '--'},
    ax=ax
)

ax.set_xlabel('P (MPa)', fontsize=10)
ax.set_ylabel('Q (MPa)', fontsize=10)
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)

# Legend
ax.legend(
    bbox_to_anchor=(1.02, 1), loc='upper left',
    fontsize=7, title='Sandstones',
    title_fontsize=8, frameon=False, ncol=1
)

plt.title('Regression: Rutter & Glover (2012) Fig 8 + Wong & Baud (2012) Fig 5a,5b', fontsize=11, pad=15)
plt.tight_layout()
plt.savefig("stats_plots/RutterGlover, 2012_Fig8 & WongBaud_2012 Fig5a,b.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# OLS Regression for new_df
X_new = rutter_glover_wong_baud['p_mpa']
y_new = rutter_glover_wong_baud['q_mpa']
X_new = sm.add_constant(X_new)  # Adds a constant term to the predictor
model_new = sm.OLS(y_new, X_new).fit()
print(model_new.summary())


In [ ]:
# Create a regression plot for limestones
plt.figure(figsize=(10, 6))
sns.scatterplot(data=wong_baud_limestone, x='p_mpa', y='q_mpa', hue='sandstone_name')
sns.regplot(data=wong_baud_limestone, x='p_mpa', y='q_mpa', scatter=False, 
            ci=None, line_kws={'color': 'black', 'linewidth': 1.0, 'linestyle': '--'})
plt.xlim(0, 500)
plt.ylim(0, 500)
plt.xlabel('P (MPa)')
plt.ylabel('Q (MPa)')
plt.legend(loc='lower right', title='Limestones')
plt.title('Regression Plot for Limestones - Wong & Baud 2012 Figure 6a,6b')
plt.savefig("stats_plots/Limestones WongBaud_2012 Fig6a,b.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# OLS Regression for limestones
X_limestone = wong_baud_limestone['p_mpa']
y_limestone = wong_baud_limestone['q_mpa']
X_limestone = sm.add_constant(X_limestone)  # Adds a constant term to the
model_limestone = sm.OLS(y_limestone, X_limestone).fit()
print(f" Wong & Baud Fig 6a,b \n{model_limestone.summary()}")

## Limestone Friction Data - Byerlee, 1978. Figure 4

In [ ]:
# filter for limestone friction data
limestone_friction_data = df_all_data[df_all_data['doi'] == '10.1007/BF00876528']

In [ ]:
# Create a regression plot for the friction data
plt.figure(figsize=(10, 6))
sns.scatterplot(data=limestone_friction_data, x='p_mpa', y='q_mpa')
sns.regplot(data=limestone_friction_data, x='p_mpa', y='q_mpa', scatter=False, 
            color='black', ci=None, label='Regression Line', 
            line_kws={'linewidth': 1.0, 'linestyle': '--'})
plt.xlim(0, 150)
plt.ylim(0, 50)
plt.xlabel('P (MPa)', fontsize=10)
plt.ylabel('Q (MPa)', fontsize=10)
plt.title('Regression Plot for Byerlee, 1978 Figure 4 Limestone Friction Data')
plt.savefig("stats_plots/Byerlee_1978_Fig4_FrictionData.png", dpi=300, bbox_inches='tight')
plt.show()

# OLS for friction data
friction_X = limestone_friction_data['p_mpa']
friction_y = limestone_friction_data['q_mpa']
friction_X = sm.add_constant(friction_X)  # Adds a constant term to the predictor
friction_model = sm.OLS(friction_y, friction_X).fit()
print(f"OLS Regression Results for Byerlee, 1978 Figure 4 Limestone Friction Data \n")
print(friction_model.summary())

In [ ]:
residuals = friction_model.resid
mse_manual = (residuals**2).mean()

print(f"Mean Squared Error (manual): {mse_manual}")


## Extract Porosity of the Sandstones

In [ ]:
# Extract Porosity of the Sandstones
def extract_porosity(s):
    """
    Extracts the porosity percentage from a string formatted like "(x%)"
    """
    matches = re.findall(r"\(([^)]+%)\)")
    if matches:
        return matches[0]
    else:
        return None

In [ ]:
# Extract Porosity of the Sandstones
def extract_porosity(s):
    matches = re.findall(r"\(([^)]+%)\)", s)
    if matches:
        return matches[0]
    else:
        return None
    
# Add a new column for porosity

rutter_glover_wong_baud['porosity'] = rutter_glover_wong_baud['sandstone_name'].apply(extract_porosity)

# Choose porosity as 18 for Berea wet
new_df_18 = rutter_glover_wong_baud.copy()
new_df_18.loc[new_df_18['porosity'] == '18 & 21%', 'porosity'] = '18%'

# Choose porosity as 21 for Berea wet
new_df_21 = rutter_glover_wong_baud.copy()
new_df_21.loc[new_df_21['porosity'] == '18 & 21%', 'porosity'] = '21%'

# Remove the percentage sign and convert to float
new_df_18['porosity'] = new_df_18['porosity'].str.replace('%', '').astype(float)
new_df_21['porosity'] = new_df_21['porosity'].str.replace('%', '').astype(float)

In [ ]:
# Range of porosity values
print("Range of porosity values using Berea wet 18:")
print(new_df_18['porosity'].describe().round(2))

print("\nRange of porosity values using Berea wet 21:")
print(new_df_21['porosity'].describe().round(2))

# Different Porosoity Plots

In [ ]:
# Split data based on median porosity

median_porosity = new_df_18['porosity'].median()

# Dataframes for porosity below and above median
below_median = new_df_18[new_df_18['porosity'] < median_porosity]
above_median = new_df_18[new_df_18['porosity'] >= median_porosity]

In [ ]:
# Sort the sandstone names for consistent ordering
sandstones_below = sorted(below_median['sandstone_name'].unique())
n_groups = len(sandstones_below)

# Pairing colors and markers
colors = sns.color_palette("colorblind", n_colors=min(n_groups, 5))
markers = ['o', 's', 'D', '^', 'v', 'P', 'X', '<', '>'] 

fig, ax = plt.subplots(figsize=(10, 6))
for i, name in enumerate(sandstones_below):
    group = below_median[below_median['sandstone_name'] == name]
    ax.scatter(
        group['p_mpa'], group['q_mpa'],
        label=name,
        color=colors[i % len(colors)],
        marker=markers[i % len(markers)],
        edgecolor='black',
        linewidth=0.4,
        s=40,
        alpha=0.8
    )
sns.regplot(
    data=below_median, x='p_mpa', y='q_mpa',
    scatter=False, ci=None,
    line_kws={'color': 'black', 'linewidth': 1.0, 'linestyle': '--'},
    ax=ax
)
ax.set_xlabel('P (MPa)', fontsize=10)
ax.set_ylabel('Q (MPa)', fontsize=10)
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)

# Legend
ax.legend(
    bbox_to_anchor=(1.02, 1), loc='upper left',
    fontsize=7, title='Sandstones',
    title_fontsize=8, frameon=False, ncol=1
)

plt.title(f'Regression: Sandstones with porosity less than median value:{median_porosity}%', fontsize=11, pad=15)
plt.tight_layout()
plt.savefig("stats_plots/porosity less than 22.8.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# OLS Regression for below median porosity
X_below = below_median['p_mpa']
y_below = below_median['q_mpa']
X_below = sm.add_constant(X_below)  # Adds a constant term to the predictor
model_below = sm.OLS(y_below, X_below).fit()
print(f"Regression Stats for Sandstones less than 22.8: \n{model_below.summary()}")

In [ ]:
# Sort the sandstone names for consistent ordering
sandstones_above = sorted(above_median['sandstone_name'].unique())
n_groups = len(sandstones_above)

# Pairing colors and markers
colors = sns.color_palette("colorblind", n_colors=min(n_groups, 5))
markers = ['o', 's', 'D', '^', 'v', 'P', 'X', '<', '>'] 

fig, ax = plt.subplots(figsize=(10, 6))
for i, name in enumerate(sandstones_above):
    group = above_median[above_median['sandstone_name'] == name]
    ax.scatter(
        group['p_mpa'], group['q_mpa'],
        label=name,
        color=colors[i % len(colors)],
        marker=markers[i % len(markers)],
        edgecolor='black',
        linewidth=0.4,
        s=40,
        alpha=0.8
    )
sns.regplot(
    data=above_median, x='p_mpa', y='q_mpa',
    scatter=False, ci=None,
    line_kws={'color': 'black', 'linewidth': 1.0, 'linestyle': '--'},
    ax=ax
)
ax.set_xlabel('P (MPa)', fontsize=10)
ax.set_ylabel('Q (MPa)', fontsize=10)
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)

# Legend
ax.legend(
    bbox_to_anchor=(1.02, 1), loc='upper left',
    fontsize=7, title='Sandstones',
    title_fontsize=8, frameon=False, ncol=1
)

plt.title(f'Regression: Sandstones with porosity higher than median value:{median_porosity}%', fontsize=11, pad=15)
plt.tight_layout()
plt.savefig("stats_plots/porosity more than 22.8.png", dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
# OLS Regression for above median porosity
X_above = above_median['p_mpa']
y_above = above_median['q_mpa']
X_above = sm.add_constant(X_above)  # Adds a constant term to the predictor
model_above = sm.OLS(y_above, X_above).fit()
print(f"Regression Stats for Sandstones higher than 22.8: \n{model_above.summary()}")